In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
import requests

In [2]:
def get_ticks(url):
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req)    
    html = BeautifulSoup(response,"html.parser")
    ticks_table = html.find(class_ = 'page')
    
    ticks = list()
    stocks = list()
    for name_box in ticks_table.find_all('a', href = True):
        stocks.append(name_box.text.strip())
        ticks.append(name_box['href'].split('-')[-1].strip())
    d = {'stock':stocks, 'tick':ticks}
    df = pd.DataFrame(data = d)
    return df

In [4]:
url = "https://www.tickertape.in/stocks"
tick_df = get_ticks(url)
tick_df.to_csv("tickers.csv")


In [5]:
def get_news(tick):
    url = "https://www.tickertape.in/stocks/" + tick + "#news"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        news_table = soup.find(class_ = 'latest-news-holder')    
        news_div = soup.find('div', {'id': 'news'})
        older_news_div = news_div.find('div', {'data-section-tag': 'olderNews'})
        news_divs = older_news_div.select('div.jsx-1359472481 div.jsx-3953764037')
        ret1 = []
        ret2 = []
        no = []
        for name_box in news_table.find_all('p', class_='shave-root'):
            ret1.append(name_box.text.strip())
        for index, news_div in enumerate(news_divs):
            title_element = news_div.select_one('h5.news-title')
            news_title = ret2.append(title_element.text.strip()) if title_element else no.append(1)
        
        return list(set(ret1)) + list(set(ret2))

In [28]:
tick = "WIPR"
name = tick_df.stock[tick_df.tick == tick].iloc[0]
ret = get_news(tick)
news = ""
for i in ret:
    news = news+i+" "

In [29]:
# from openai import OpenAI

# openai = OpenAI(
#     api_key="bPdeUDQV6WUDGE6ifWQHult7iii7OWFj",
#     base_url="https://api.deepinfra.com/v1/openai",
# )

# prompt = f"Consider yourself a Financial Analyst. You will be given a set of recent news about the stock {name}, ignore the news that might not be relevant. You need to provide a summary of what all the news articles are trying to say and then give a suggestion about the health of the stock. At the end of the analysis, give the stock a score out of 10 based on the health of the stock. The format for score is: Score==<score>"

# chat_completion = openai.chat.completions.create(
#     model="mistralai/Mistral-7B-Instruct-v0.1",
#     messages=[
#         {"role": "system", "content": news},
#         {"role": "user", "content": prompt},
#     ],
#     temperature=0.5
# )

# print(chat_completion.choices[0].message.content)


 Summary:

* Wipro's stock price went up despite a decline in net profit in Q3.
* Wipro's future growth strategy was discussed by Rishad Premji, addressing leadership concerns.
* HDFC Bank and Wipro ADRs plunged.
* India Market Open.
* NDTV Profit LIVE.
* Wipro CEO Thierry Delaporte talked about achievements and targets.
* Wipro Q3 earnings beat expectations but analysts are still in 'wait and watch' mode.
* Wipro says it sees green shoots in demand, but discretionary spend is still elusive.
* Wipro CEO Thierry Delaporte is aligned with the board on targets and achievements.
* Wipro's stock jumped 10% after Q3 earnings signaled recovery in tech demand.

Suggestion:

Based on the news articles, it seems that Wipro's Q3 earnings were better than expected, and the company is optimistic about its future growth. However, there are concerns about discretionary spend, which may impact the company's growth in the short term. Overall, the news suggests that Wipro is in a good position, and its 